### Logistic Regression on Amzon Fine Food Reviews

DataSet Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

In [1]:
#Importing all Necessory Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
import warnings
warnings.filterwarnings("ignore")

C:\Users\arjun\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Loding Bow data.
import pickle
with open('train_bow.pickle','rb') as handle:
    train_bow = pickle.load(handle)
with open('test_bow.pickle','rb') as handle:
    test_bow = pickle.load(handle)

In [3]:
#Loding Tfidf data.
import pickle
with open('train_tfidf.pickle','rb') as handle:
    train_tfidf = pickle.load(handle)
with open('test_tfidf.pickle','rb') as handle:
    test_tfidf = pickle.load(handle)

In [4]:
#Loding Avg word2vec
import pickle
with open('train_avg_word2vec.pickle','rb') as handle:
    train_avg_word2vec = pickle.load(handle)
with open('test_avg_word2vec.pickle','rb') as handle:
    test_avg_word2vec = pickle.load(handle)

In [5]:
#Loding Tfidf Word2Vec.
import pickle
with open('train_tfidf_word2vec.pickle','rb') as handle:
    train_tfidf_word2vec = pickle.load(handle)
with open('test_tfidf_word2vec.pickle','rb') as handle:
    test_tfidf_word2vec = pickle.load(handle)

In [6]:
#This is ouput is for Bow and Tfidf
with open('y_train.pickle','rb') as handle:
    y_train = pickle.load(handle)
with open('y_test.pickle','rb') as handle:
    y_test = pickle.load(handle)

In [7]:
#This is for Avg word2vec and tfidf word2vec
with open('y_train_w.pickle','rb') as handle:
    y_train_w = pickle.load(handle)
with open('y_test_w.pickle','rb') as handle:
    y_test_w = pickle.load(handle)

In [8]:
from sklearn.preprocessing import normalize
#Feature Normalization.
#Bow features
train_bow_normalize = normalize(train_bow, axis=0)
test_bow_normalize = normalize(test_bow, axis=0)
#Tfidf Features
train_tfidf_normalize = normalize(train_tfidf, axis=0)
test_tfidf_normalize = normalize(test_tfidf, axis=0)
#Avg word2Vec Features
train_avgw2v_normalize = normalize(train_avg_word2vec, axis=0)
test_avgw2v_normalize = normalize(test_avg_word2vec, axis=0)
#Tfidf Weighted Word2Vec
train_tfidfw2v_normalize = normalize(train_tfidf_word2vec, axis=0)
test_tfidfw2v_normalize = normalize(test_tfidf_word2vec, axis=0)

### Featurization: Bag of Words

#### 1. GridSearchCV_L2-Reg

In [9]:
#Logistic Regressiojn with Grid search cv L2 regularization.
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
tuned_parameters = [{'alpha':[10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4]}]
model = GridSearchCV(SGDClassifier(class_weight='balanced', penalty='l2', loss='log', random_state=42), tuned_parameters, scoring='accuracy')
model.fit(train_bow_normalize, y_train)
print(model.best_estimator_)
print(model.score(test_bow_normalize, y_test))

SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.874


#### 2. GridSearchCV_L1-Reg

In [10]:
# Logistic Regression with GridSearchCV L-1 Regulizor.
model = GridSearchCV(SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42), tuned_parameters, scoring='accuracy')
model.fit(train_bow_normalize, y_train)
print(model.best_estimator_)
print(model.score(test_bow_normalize, y_test))

SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8553333333333333


In [11]:
#Look at how Sparsity and performance changing for the given C(here alpha).
LR = SGDClassifier(class_weight='balanced', penalty='l2', loss='log', random_state=42, alpha=0.01)
LR.fit(train_bow_normalize, y_train)
w = LR.coef_
print(LR.score(test_bow_normalize, y_test))
print(np.count_nonzero(w))

0.863
12057


In [12]:
LR = SGDClassifier(class_weight='balanced', penalty='l2', loss='log', random_state=42, alpha=0.1)
LR.fit(train_bow_normalize, y_train)
w = LR.coef_
print(LR.score(test_bow_normalize, y_test))
print(np.count_nonzero(w))

0.17033333333333334
12057


In [13]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=1.0)
LR.fit(train_bow_normalize, y_train)
w = LR.coef_
print(LR.score(test_bow_normalize, y_test))
print(np.count_nonzero(w))

0.17033333333333334
0


In [14]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=10)
LR.fit(train_bow_normalize, y_train)
w = LR.coef_
print(LR.score(test_bow_normalize, y_test))
print(np.count_nonzero(w))

0.8296666666666667
0


In [15]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=100)
LR.fit(train_bow_normalize, y_train)
w = LR.coef_
print(LR.score(test_bow_normalize, y_test))
print(np.count_nonzero(w))

0.8296666666666667
0


In [16]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=1000)
LR.fit(train_bow_normalize, y_train)
w = LR.coef_
print(LR.score(test_bow_normalize, y_test))
print(np.count_nonzero(w))

0.8296666666666667
0


#### Observation:

* 'C' is inversly proportional to 'lamda'.
* As 'C' increases, no of non zero's in w will reduces.

#### 3. RandomizedSearchCV_L2-Reg

In [17]:
#LogisticRegression with RandamizedSearchCV, Default:L2 reg
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
#tuned_parameters = {'C':[1,10**4]}
param_dist = {'alpha':randint(10**-3,10**3)}
model = RandomizedSearchCV(SGDClassifier(class_weight='balanced', penalty='l2', loss='log', random_state=42), param_distributions = param_dist, scoring='accuracy')
model.fit(train_bow_normalize, y_train)
print(model.best_estimator_)
print(model.score(test_bow_normalize, y_test))

SGDClassifier(alpha=320, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


#### 4. RandomizedSearchCV_L1-Reg

In [18]:
#LogisticRegression with RandamizedSearchCV, Default:L2 reg
from scipy.stats import randint
#tuned_parameters = {'C':[10**0, 10**4]}
param_dist = {'alpha': randint(10**-3,10**3)}
model = RandomizedSearchCV(SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42), param_distributions = param_dist, scoring='accuracy')
model.fit(train_bow_normalize, y_train)
print(model.best_estimator_)
print(model.score(test_bow_normalize, y_test))

SGDClassifier(alpha=654, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


#### 5. Feature Importance

In [19]:
#Finding weights before applying noise.
log_reg_bow_before = SGDClassifier(alpha=0.0001, class_weight='balanced', penalty='l2', loss='log', random_state=42)
log_reg_bow_before.fit(train_bow_normalize, y_train)
#print(log_reg_bow_before.coef_)
#--------------------------------------------------------------
print(train_bow_normalize.shape)
print(test_bow_normalize.shape)
#---------------------------------------------------------------
#Creating a noise matrix and addingup both,
mu, sigma = 0, 0.1 
# creating a noise with the same dimension as the dataset 
train_noise = np.random.normal(mu, sigma, [7000,12057])
train_data_noise = train_bow_normalize + train_noise
#-----------------------------------------------------------------
#Finding weights before applying noise.
log_reg_bow_after = SGDClassifier(alpha=0.0001, class_weight='balanced', penalty='l2', loss='log', random_state=42)
log_reg_bow_after.fit(train_data_noise, y_train)
#print(log_reg_bow_after.coef_)

(7000, 12057)
(3000, 12057)


SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [20]:
diff_coef = (log_reg_bow_before.coef_)-(log_reg_bow_after.coef_)
#print(diff_coef)
div_coef = diff_coef/(log_reg_bow_before.coef_)
per_coef = abs(div_coef) * 100
per_mean = np.mean(per_coef)
print(per_mean)

264594.42503003316


** As weight vector values before and after pertubation changes significantly, then we can't use |w| as feature importance measure.**

In [21]:
# Feature Importance with Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
# feature extraction
model = ExtraTreesClassifier()
model.fit(train_bow_normalize, y_train)
feat_imp = list(model.feature_importances_) #Gives the feature importance scores.
print(type(feat_imp))
sort_feat_imp = sorted(feat_imp, reverse=True)
top_feat_imp = sort_feat_imp[:5]
print(top_feat_imp)

<class 'list'>
[0.008846311487409594, 0.0077703992687845865, 0.007452338787385092, 0.006707367381649398, 0.005970118287668688]


In [22]:
feat_imp.index(0.006707367381649398)

3507

### Featurization: Tfidf

#### 1. GridSearchCV_L2_Reg

In [23]:
#Splitting data into train and test
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

tuned_parameters = [{'alpha':[10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4]}]

model = GridSearchCV(SGDClassifier(class_weight='balanced', penalty='l2', loss='log', random_state=42), tuned_parameters, scoring='accuracy')
model.fit(train_tfidf_normalize, y_train)
print(model.best_estimator_)
print(model.score(test_tfidf_normalize, y_test))

SGDClassifier(alpha=0.001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8673333333333333


#### 2. GridSearchCV_L1_Reg

In [24]:
# Logistic Regression with GridSearchCV L-1 Regulizor.
model = GridSearchCV(SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42), tuned_parameters, scoring='accuracy')
model.fit(train_tfidf_normalize, y_train)
print(model.best_estimator_)
print(model.score(test_tfidf_normalize, y_test))

SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.83


In [25]:
#Look at how Sparsity and performance changing for the given C.
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=0.01)
LR.fit(train_tfidf_normalize, y_train)
w = LR.coef_
print(LR.score(test_tfidf_normalize, y_test))
print(np.count_nonzero(w))

0.8296666666666667
0


In [26]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=0.1)
LR.fit(train_tfidf_normalize, y_train)
w = LR.coef_
print(LR.score(test_tfidf_normalize, y_test))
print(np.count_nonzero(w))

0.17033333333333334
0


In [27]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=1.0)
LR.fit(train_tfidf_normalize, y_train)
w = LR.coef_
print(LR.score(test_tfidf_normalize, y_test))
print(np.count_nonzero(w))

0.17033333333333334
0


In [28]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=10)
LR.fit(train_tfidf_normalize, y_train)
w = LR.coef_
print(LR.score(test_tfidf_normalize, y_test))
print(np.count_nonzero(w))

0.8296666666666667
0


In [29]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=100)
LR.fit(train_tfidf_normalize, y_train)
w = LR.coef_
print(LR.score(test_tfidf_normalize, y_test))
print(np.count_nonzero(w))

0.8296666666666667
0


In [30]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=1000)
LR.fit(train_tfidf_normalize, y_train)
w = LR.coef_
print(LR.score(test_tfidf_normalize, y_test))
print(np.count_nonzero(w))

0.8296666666666667
0


**Obsevation:**
* As 'C' increases, no of non zero's in w will reduces.

#### 3. RandomizedSearchCV_L2_Reg

In [31]:
#LogisticRegression with RandamizedSearchCV, Default:L2 reg
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
#tuned_parameters = {'C':[1,10**4]}
param_dist = {'alpha':randint(10**-3,10**3)}
model = RandomizedSearchCV(SGDClassifier(class_weight='balanced', penalty='l2', loss='log', random_state=42), param_distributions = param_dist, scoring='accuracy')
model.fit(train_tfidf_normalize, y_train)
print(model.best_estimator_)
print(model.score(test_tfidf_normalize, y_test))

SGDClassifier(alpha=982, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


#### 4. RandomizedSearchCV_L1_Reg

In [32]:
#LogisticRegression with RandamizedSearchCV, Default:L2 reg
from scipy.stats import randint
#tuned_parameters = {'C':[10**0, 10**4]}
param_dist = {'alpha': randint(10**-3,10**3)}
model = RandomizedSearchCV(SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42), param_distributions = param_dist, scoring='accuracy')
model.fit(train_tfidf_normalize, y_train)
print(model.best_estimator_)
print(model.score(test_tfidf_normalize, y_test))

SGDClassifier(alpha=106, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


#### 5. Feature Importance

In [33]:
#Finding weights before applying noise.
log_reg_tfidf_before = SGDClassifier(alpha=0.001, class_weight='balanced', penalty='l2', loss='log', random_state=42)
log_reg_tfidf_before.fit(train_bow_normalize, y_train)
#print(log_reg_tfidf_before.coef_)
#--------------------------------------------------------------
print(train_tfidf_normalize.shape)
#print(test_tfidf_normalize.shape)
#---------------------------------------------------------------
#Creating a noise matrix and addingup both,
mu, sigma = 0, 0.1 
# creating a noise with the same dimension as the dataset 
train_noise = np.random.normal(mu, sigma, [7000,12057])
train_data_noise = train_tfidf_normalize + train_noise
#-----------------------------------------------------------------
#Finding weights before applying noise.
log_reg_tfidf_after = SGDClassifier(alpha=0.001, class_weight='balanced', penalty='l2', loss='log', random_state=42)
log_reg_tfidf_after.fit(train_data_noise, y_train)
#print(log_reg_tfidf_after.coef_)

(7000, 12057)


SGDClassifier(alpha=0.001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [34]:
diff_coef = (log_reg_tfidf_before.coef_)-(log_reg_tfidf_after.coef_)
#print(diff_coef)
div_coef = diff_coef/(log_reg_tfidf_before.coef_)
per_coef = abs(div_coef) * 100
per_mean = np.mean(per_coef)
print(per_mean)

792.7183202856952


** As weight vector values before and after pertubation changes significantly, then we can't use |w| as feature importance measure.**

In [54]:
# Feature Importance with Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
# feature extraction
model = ExtraTreesClassifier()
model.fit(train_tfidf_normalize, y_train)
feat_imp = list(model.feature_importances_) #Gives the feature importance scores.
print(type(feat_imp))
sort_feat_imp = sorted(feat_imp, reverse=True)
top_feat_imp = sort_feat_imp[:5]
print(top_feat_imp)

<class 'list'>
[0.012840489354814438, 0.011012369740903407, 0.007936368564732798, 0.007655503374802364, 0.007388203092251884]


In [55]:
feat_imp.index(0.007388203092251884)

11867

### Featurization: Avg Word2Vec

#### 1. GridSearchCV_L2_Reg

In [37]:
#Splitting data into train and test
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier

tuned_parameters = [{'alpha':[10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4]}]

model = GridSearchCV(SGDClassifier(class_weight='balanced', penalty='l2', loss='log', random_state=42), tuned_parameters, scoring='accuracy')
model.fit(train_avgw2v_normalize, y_train_w)
print(model.best_estimator_)
print(model.score(test_avgw2v_normalize, y_test_w))

SGDClassifier(alpha=100, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


#### 2. GridSearchCV_L1_Reg

In [38]:
# Logistic Regression with GridSearchCV L-1 Regulizor.
model = GridSearchCV(SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42), tuned_parameters, scoring='accuracy')
model.fit(train_avgw2v_normalize, y_train_w)
print(model.best_estimator_)
print(model.score(test_avgw2v_normalize, y_test_w))

SGDClassifier(alpha=100, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


In [39]:
#Look at how Sparsity and performance changing for the given C or alpha.
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=0.01)
LR.fit(train_avgw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_avgw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.17033333333333334
0


In [40]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=0.1)
LR.fit(train_avgw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_avgw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.17033333333333334
0


In [41]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=1)
LR.fit(train_avgw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_avgw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.17033333333333334
0


In [42]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=10)
LR.fit(train_avgw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_avgw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.8296666666666667
0


In [43]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=100)
LR.fit(train_avgw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_avgw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.8296666666666667
0


In [44]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=1000)
LR.fit(train_avgw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_avgw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.8296666666666667
0


**Observation:**
* As 'C' increases, no of non zero's in w will reduces.

#### 3. RandomizedSearchCV_L2_Reg

In [45]:
#LogisticRegression with RandamizedSearchCV, Default:L2 reg
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
#tuned_parameters = {'C':[1,10**4]}
param_dist = {'alpha':randint(10**-3,10**3)}
model = RandomizedSearchCV(SGDClassifier(class_weight='balanced', penalty='l2', loss='log', random_state=42), param_distributions = param_dist, scoring='accuracy')
model.fit(train_avgw2v_normalize, y_train_w)
print(model.best_estimator_)
print(model.score(test_avgw2v_normalize, y_test_w))

SGDClassifier(alpha=220, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


#### 4. RandomizedSearchCV_L1_Reg

In [46]:
#LogisticRegression with RandamizedSearchCV, Default:L2 reg
from scipy.stats import randint
#tuned_parameters = {'C':[10**0, 10**4]}
param_dist = {'alpha': randint(10**-3,10**3)}
model = RandomizedSearchCV(SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42), param_distributions = param_dist, scoring='accuracy')
model.fit(train_avgw2v_normalize, y_train_w)
print(model.best_estimator_)
print(model.score(test_avgw2v_normalize, y_test_w))

SGDClassifier(alpha=792, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


#### 5. Feature Importance

In [73]:
#Finding weights before applying noise.
log_reg_avgw2v_before = SGDClassifier(alpha=0.001, class_weight='balanced', penalty='l2', loss='log', random_state=42)
log_reg_avgw2v_before.fit(train_avgw2v_normalize, y_train_w)
#print(log_reg_avgw2v_before.coef_)
#--------------------------------------------------------------
print(train_avgw2v_normalize.shape)
#print(test_avgw2v_normalize.shape)
#---------------------------------------------------------------
#Creating a noise matrix and addingup both,
mu, sigma = 0, 0.1 
# creating a noise with the same dimension as the dataset 
train_noise = np.random.normal(mu, sigma, [7000,300])
train_data_noise = train_avgw2v_normalize + train_noise
#-----------------------------------------------------------------
#Finding weights before applying noise.
log_reg_avgw2v_after = SGDClassifier(alpha=0.001, class_weight='balanced', penalty='l2', loss='log', random_state=42)
log_reg_avgw2v_after.fit(train_data_noise, y_train_w)
#print(log_reg_avgw2v_after.coef_)

(7000, 300)


SGDClassifier(alpha=0.001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [74]:
diff_coef = (log_reg_avgw2v_before.coef_)-(log_reg_avgw2v_after.coef_)
#print(diff_coef)
div_coef = diff_coef/(log_reg_avgw2v_before.coef_)
per_coef = abs(div_coef) * 100
per_mean = np.mean(per_coef)
print(per_mean)

2846.572409658388


** As weight vector values before and after pertubation changes significantly, then we can't use |w| as feature importance measure.**

In [75]:
# Feature Importance with Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
# feature extraction
model = ExtraTreesClassifier()
model.fit(train_avgw2v_normalize, y_train_w)
feat_imp = list(model.feature_importances_) #Gives the feature importance scores.
print(type(feat_imp))
sort_feat_imp = sorted(feat_imp, reverse=True)
top_feat_imp = sort_feat_imp[:5]
print(top_feat_imp)

<class 'list'>
[0.008786456609959808, 0.007474162518743816, 0.007096683496718841, 0.006863860288569282, 0.0058719100446479894]


In [76]:
feat_imp.index(0.0058719100446479894)

135

### Featurization: Tfidf Weighted Word2Vec

#### 1. GridSearchCV_L2_Reg

In [56]:
#Splitting data into train and test
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier

tuned_parameters = [{'alpha':[10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4]}]

model = GridSearchCV(SGDClassifier(class_weight='balanced', penalty='l2', loss='log', random_state=42), tuned_parameters, scoring='accuracy')
model.fit(train_tfidfw2v_normalize, y_train_w)
print(model.best_estimator_)
print(model.score(test_tfidfw2v_normalize, y_test_w))

SGDClassifier(alpha=100, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


#### 2. GridSearchCV_L1_Reg

In [57]:
# Logistic Regression with GridSearchCV L-1 Regulizor.
model = GridSearchCV(SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42), tuned_parameters, scoring='accuracy')
model.fit(train_tfidfw2v_normalize, y_train_w)
print(model.best_estimator_)
print(model.score(test_tfidfw2v_normalize, y_test_w))

SGDClassifier(alpha=100, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


In [58]:
#Look at how Sparsity and performance changing for the given C or alpha.
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=0.01)
LR.fit(train_tfidfw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_tfidfw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.17033333333333334
0


In [59]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=0.1)
LR.fit(train_tfidfw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_tfidfw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.17033333333333334
0


In [60]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=1)
LR.fit(train_tfidfw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_tfidfw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.17033333333333334
0


In [61]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=10)
LR.fit(train_tfidfw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_tfidfw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.8296666666666667
0


In [62]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=100)
LR.fit(train_tfidfw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_tfidfw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.8296666666666667
0


In [63]:
LR = SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42, alpha=1000)
LR.fit(train_tfidfw2v_normalize, y_train_w)
w = LR.coef_
print(LR.score(test_tfidfw2v_normalize, y_test_w))
print(np.count_nonzero(w))

0.8296666666666667
0


**Observation:**
* As 'C' increases, no of non zero's in w will reduces.

#### 3. RandomizedSearchCV_L2_Reg

In [64]:
#LogisticRegression with RandamizedSearchCV, Default:L2 reg
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
#tuned_parameters = {'C':[1,10**4]}
param_dist = {'alpha':randint(10**-3,10**3)}
model = RandomizedSearchCV(SGDClassifier(class_weight='balanced', penalty='l2', loss='log', random_state=42), param_distributions = param_dist, scoring='accuracy')
model.fit(train_tfidfw2v_normalize, y_train_w)
print(model.best_estimator_)
print(model.score(test_tfidfw2v_normalize, y_test_w))

SGDClassifier(alpha=748, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


#### 4. RandomizedSearchCV_L1_Reg

In [65]:
#LogisticRegression with RandamizedSearchCV, Default:L2 reg
from scipy.stats import randint
#tuned_parameters = {'C':[10**0, 10**4]}
param_dist = {'alpha': randint(10**-3,10**3)}
model = RandomizedSearchCV(SGDClassifier(class_weight='balanced', penalty='l1', loss='log', random_state=42), param_distributions = param_dist, scoring='accuracy')
model.fit(train_tfidfw2v_normalize, y_train_w)
print(model.best_estimator_)
print(model.score(test_tfidfw2v_normalize, y_test_w))

SGDClassifier(alpha=318, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)
0.8296666666666667


#### 5. Feature Importance

In [69]:
#Finding weights before applying noise.
log_reg_tfidfw2v_before = SGDClassifier(alpha=0.001, class_weight='balanced', penalty='l2', loss='log', random_state=42)
log_reg_tfidfw2v_before.fit(train_tfidfw2v_normalize, y_train_w)
#print(log_reg_tfidfw2v_before.coef_)
#--------------------------------------------------------------
print(train_tfidfw2v_normalize.shape)
print(test_tfidfw2v_normalize.shape)
#---------------------------------------------------------------
#Creating a noise matrix and addingup both,
mu, sigma = 0, 0.1 
# creating a noise with the same dimension as the dataset 
train_noise = np.random.normal(mu, sigma, [7000,300])
train_data_noise = train_tfidfw2v_normalize + train_noise
#-----------------------------------------------------------------
#Finding weights before applying noise.
log_reg_tfidfw2v_after = SGDClassifier(alpha=0.001, class_weight='balanced', penalty='l2', loss='log', random_state=42)
log_reg_tfidfw2v_after.fit(train_data_noise, y_train_w)
#print(log_reg_tfidfw2v_after.coef_)

(7000, 300)
(3000, 300)


SGDClassifier(alpha=0.001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [70]:
diff_coef = (log_reg_tfidfw2v_before.coef_)-(log_reg_tfidfw2v_after.coef_)
#print(diff_coef)
div_coef = diff_coef/(log_reg_tfidfw2v_before.coef_)
per_coef = abs(div_coef) * 100
per_mean = np.mean(per_coef)
print(per_mean)

292.1420790948361


** As weight vector values before and after pertubation changes significantly, then we can't use |w| as feature importance measure.**

In [71]:
# Feature Importance with Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
# feature extraction
model = ExtraTreesClassifier()
model.fit(train_tfidfw2v_normalize, y_train_w)
feat_imp = list(model.feature_importances_) #Gives the feature importance scores.
print(type(feat_imp))
sort_feat_imp = sorted(feat_imp, reverse=True)
top_feat_imp = sort_feat_imp[:5]
print(top_feat_imp)

<class 'list'>
[0.008784125106847087, 0.006463625321741884, 0.006302503127649487, 0.0055995824361515945, 0.005416648794072623]


In [72]:
feat_imp.index(0.005416648794072623)

66

#### Results:

* Bow----------------------->alpha=0.0001-------------->Accuracy:87.40%
* Tfidf----------------------->alpha=0.001---------------->Accuracy:86.73%
* Avg Word2Vec--------->alpha=100------------------->Accuracy:82.96%
* Tfidf Word2Vec--------->alpha=100------------------>Accuracy:82.96%
* Among all the featurization techniques, we are getting slightly better accuracy with Bag of words featurization
* Logistic regression works very well for the higher dimensional data.